In [14]:
import os
import typing
from sklearn.gaussian_process.kernels import *
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

from autograd import grad 
import tensorflow as tf
from keras.losses import categorical_crossentropy, mean_squared_error
from keras import layers
from keras.layers import *
from keras.metrics import AUC
from keras.callbacks import EarlyStopping
from keras.activations import *

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time

from tqdm import tqdm

from flaml import AutoML

import eli5
from eli5.sklearn import PermutationImportance

import shap

In [15]:
# Cost function constants
COST_W_UNDERPREDICT = 25.0
COST_W_NORMAL = 1.0
COST_W_OVERPREDICT = 10.0

In [16]:
def cost_function(ground_truth: np.ndarray, predictions: np.ndarray) -> float:
    # Unweighted cost
    cost = (ground_truth - predictions) ** 2
    weights = np.ones_like(cost) * COST_W_NORMAL

    # Case i): underprediction
    mask_1 = predictions < ground_truth
    weights[mask_1] = COST_W_UNDERPREDICT

    # Case ii): significant overprediction
    mask_2 = (predictions >= 1.2*ground_truth)
    weights[mask_2] = COST_W_OVERPREDICT

    # Weigh the cost and return the average
    return np.mean(cost * weights)


In [17]:
#convert the cost function to a tensorflow loss function
def cost_function_tf(): 
  def cost_fn(y_true, y_pred):
    # Unweighted cost
    cost = (y_true - y_pred) ** 2
    weights = tf.ones_like(cost) * COST_W_NORMAL

    # Case i): underprediction
    mask_1 = tf.cast(y_pred < y_true, tf.float32)
    weights = weights + mask_1 * (COST_W_UNDERPREDICT - COST_W_NORMAL)

    # Case ii): significant overprediction
    mask_2 = tf.cast(y_pred >= 1.2*y_true, tf.float32)
    weights = weights + mask_2 * (COST_W_OVERPREDICT - COST_W_NORMAL)

    # Weigh the cost and return the average
    return tf.reduce_mean(cost * weights)
    
  return cost_fn


In [22]:
# Load the training dateset and test features
X = np.loadtxt('train_x.csv', delimiter=',', skiprows=1)
y = np.loadtxt('train_y.csv', delimiter=',', skiprows=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

model = tf.keras.Sequential([
    layers.Input(shape=(X.shape[1],)),
    layers.Dense(2048, activation=relu),
    # layers.Dropout(0.1),
    layers.Dense(2048, activation=relu),
    layers.Dense(2048, activation=relu),
    layers.Dense(2048, activation=relu),
    layers.Dense(2048, activation=relu),
    layers.Dense(1, activation='linear')
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=cost_function_tf(), metrics=[cost_function_tf()])
model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test, y_test), verbose=1)

Epoch 1/100
95/95 [==============================] - 2s 12ms/step - loss: 5642.0737 - cost_fn: 5640.5356 - val_loss: 3731.8118 - val_cost_fn: 3731.9402
Epoch 2/100
95/95 [==============================] - 1s 10ms/step - loss: 3793.6536 - cost_fn: 3793.0076 - val_loss: 3776.5408 - val_cost_fn: 3775.2312
Epoch 3/100
95/95 [==============================] - 1s 10ms/step - loss: 3810.4836 - cost_fn: 3810.0168 - val_loss: 3941.7327 - val_cost_fn: 3941.8333
Epoch 4/100
95/95 [==============================] - 1s 10ms/step - loss: 3749.4243 - cost_fn: 3749.0559 - val_loss: 3630.5850 - val_cost_fn: 3630.4121
Epoch 5/100
95/95 [==============================] - 1s 10ms/step - loss: 3650.7690 - cost_fn: 3650.6648 - val_loss: 3521.0935 - val_cost_fn: 3520.1257
Epoch 6/100
95/95 [==============================] - 1s 11ms/step - loss: 3457.3506 - cost_fn: 3457.2639 - val_loss: 3283.0530 - val_cost_fn: 3282.7283
Epoch 7/100
95/95 [==============================] - 1s 10ms/step - loss: 3445.8853 - co